### Analysis and Prediction on Data :Defaulted Credit Card Clients

In [21]:

#Import Libraries
library(mlr)           # ML toolkit
library(caret)         # ML toolkit
library(nnet)          # class.ind() function
library(neuralnet)     # Deep Neural Networks
library(LiblineaR)     # LR Lasso (l1)
library(randomForest)  # Random Forest
library(adabag)        # Boosting
library(e1071)         # SVM
library(ggplot2)       # Visualization
library(plotly)        # 3D visualization

# Resize plot
library(repr)  # String and binary representations

In [22]:
setwd("C:\\Users\\dwijayaweera\\Documents\\2nd STAT &MIL for marketing\\SML_Section7_v2.1_with_solution\\SML_Section7_v2.1\\data\\com1_default")


In [23]:
# Read data
defaultedCreditCards <- read.csv('default.csv', sep=';')


In [24]:
head(defaultedCreditCards)

X,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0


In [25]:
str(defaultedCreditCards)

'data.frame':	30000 obs. of  25 variables:
 $ X  : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X1 : int  20000 120000 90000 50000 50000 50000 500000 100000 140000 20000 ...
 $ X2 : int  2 2 2 2 1 1 1 2 2 1 ...
 $ X3 : int  2 2 2 2 2 1 1 2 3 3 ...
 $ X4 : int  1 2 2 1 1 2 2 2 1 2 ...
 $ X5 : int  24 26 34 37 57 37 29 23 28 35 ...
 $ X6 : int  2 -1 0 0 -1 0 0 0 0 -2 ...
 $ X7 : int  2 2 0 0 0 0 0 -1 0 -2 ...
 $ X8 : int  -1 0 0 0 -1 0 0 -1 2 -2 ...
 $ X9 : int  -1 0 0 0 0 0 0 0 0 -2 ...
 $ X10: int  -2 0 0 0 0 0 0 0 0 -1 ...
 $ X11: int  -2 2 0 0 0 0 0 -1 0 -1 ...
 $ X12: int  3913 2682 29239 46990 8617 64400 367965 11876 11285 0 ...
 $ X13: int  3102 1725 14027 48233 5670 57069 412023 380 14096 0 ...
 $ X14: int  689 2682 13559 49291 35835 57608 445007 601 12108 0 ...
 $ X15: int  0 3272 14331 28314 20940 19394 542653 221 12211 0 ...
 $ X16: int  0 3455 14948 28959 19146 19619 483003 -159 11793 13007 ...
 $ X17: int  0 3261 15549 29547 19131 20024 473944 567 3719 13912 ...
 $ X18: int  0 0 1518 20

In [26]:
table(defaultedCreditCards$Y)


    0     1 
23364  6636 

In [27]:
dim(defaultedCreditCards)

[1] 30000    25

### Q1 Randomly divide data into train/test as 80/20 (set.seed=1).

In [28]:
# The general train and test split
set.seed(1)
train_idx <- sample(1:nrow(defaultedCreditCards), round(nrow(defaultedCreditCards)*0.8))
train <- defaultedCreditCards[train_idx, ]
test <- defaultedCreditCards[-train_idx, ]


dim(train)
dim(test)

[1] 24000    25

[1] 6000   25

In [29]:
#Creating dummy variables from the target variable
DCC2 <- cbind(defaultedCreditCards[, 2:(ncol(defaultedCreditCards)-1)], class.ind(as.factor(defaultedCreditCards$Y)))
names(DCC2) <- c(names(DCC2)[1:(ncol(DCC2)-2)], "No", "Yes")

In [30]:
str(DCC2)

'data.frame':	30000 obs. of  25 variables:
 $ X1 : int  20000 120000 90000 50000 50000 50000 500000 100000 140000 20000 ...
 $ X2 : int  2 2 2 2 1 1 1 2 2 1 ...
 $ X3 : int  2 2 2 2 2 1 1 2 3 3 ...
 $ X4 : int  1 2 2 1 1 2 2 2 1 2 ...
 $ X5 : int  24 26 34 37 57 37 29 23 28 35 ...
 $ X6 : int  2 -1 0 0 -1 0 0 0 0 -2 ...
 $ X7 : int  2 2 0 0 0 0 0 -1 0 -2 ...
 $ X8 : int  -1 0 0 0 -1 0 0 -1 2 -2 ...
 $ X9 : int  -1 0 0 0 0 0 0 0 0 -2 ...
 $ X10: int  -2 0 0 0 0 0 0 0 0 -1 ...
 $ X11: int  -2 2 0 0 0 0 0 -1 0 -1 ...
 $ X12: int  3913 2682 29239 46990 8617 64400 367965 11876 11285 0 ...
 $ X13: int  3102 1725 14027 48233 5670 57069 412023 380 14096 0 ...
 $ X14: int  689 2682 13559 49291 35835 57608 445007 601 12108 0 ...
 $ X15: int  0 3272 14331 28314 20940 19394 542653 221 12211 0 ...
 $ X16: int  0 3455 14948 28959 19146 19619 483003 -159 11793 13007 ...
 $ X17: int  0 3261 15549 29547 19131 20024 473944 567 3719 13912 ...
 $ X18: int  0 0 1518 2000 2000 2500 55000 380 3329 0 ...
 $ X

In [31]:
#dimensions
dim(DCC2)

[1] 30000    25

In [ ]:
# Create a sample of the data
set.seed(1)
DCC2 <- DCC2[sample(1:nrow(DCC2), round(nrow(DCC2)*0.05)), ]

# Split the adjusted data for NN model into train/test

train_idx <- sample(1:nrow(DCC2), round(nrow(DCC2)*0.8))
train_df2 <- DCC2[train_idx, ]
test_df2 <- DCC2[-train_idx, ]


dim(train_df2)
dim(test_df2)

In [ ]:


#create the formula


#DCC_formula <- as.formula("N + Y ~ X1+ X2+ X3+ X4+ X5+ X6+ X7+ X8+X9+X10+X11+X12+X13+X14+X15+X16+X17+X18+X19+X20+X21+X22+X23")

DCC_formula <- as.formula(paste0('No + Yes ~ ', paste(names(DCC2)[1:(ncol(DCC2)-2)], collapse=' + ')))


### Q2: Build a NN model with 1 hidden layer of 30 neurons, sigmoid activation function. 

In [ ]:

# Fit the Neural Network model
dcc_nnet_i <- neuralnet(DCC_formula,
                     train_df2,
                     hidden=c(10, 10, 10),        # Size of the hidden layers
                     
                     rep=1,                 # Number of training repeat
                     lifesign='full',       # Print during train
                     algorithm='backprop',  # Algorithm to calculate the network
                     learningrate=0.01,     # Learning rate
                     err.fct='ce',          # Error function, cross-entropy
                     act.fct="logistic",    # Function use to calculate the result
                     linear.output=F
)

In [ ]:
# Make prediction and evaluation on train
y_train_pred <- predict(dcc_nnet_i, train[, c(1:24)])
mean((max.col(y_train_pred)-1) == train$Y)

In [ ]:
# Make prediction and evaluation on test
y_test_pred <- predict(dcc_nnet_i, test[, c(1:24)])
mean((max.col(y_test_pred)-1) == test$Y)

### Q3  Build a deep NN model with multiple hidden layers (of your choice) and sigmoid activation function. 

In [ ]:

# Fit the Neural Network model
dcc_nnet_ii <- neuralnet(DCC_formula,
                     train_df2,
                     hidden=c(5, 5, 5),        # Size of the hidden layers
                     hidden=c(5, 5, 5),        # Size of the hidden layers
                     hidden=c(10, 10, 10),        # Size of the hidden layers
                     stepmax=20000,        # Maximum training step
                     rep=1,                 # Number of training repeat
                     lifesign='full',       # Print during train
                     algorithm='backprop',  # Algorithm to calculate the network
                     learningrate=0.01,     # Learning rate
                     err.fct='ce',          # Error function, cross-entropy
                     act.fct="logistic",    # Function use to calculate the result
                     linear.output=F
)

In [ ]:
# Make prediction and evaluation on train
y_train_pred <- predict(dcc_nnet_ii, train[, c(1:24)])
mean((max.col(y_train_pred)-1) == train$Y)

In [ ]:
# Make prediction and evaluation on test
y_test_pred <- predict(dcc_nnet_ii, test[, c(1:24)])
mean((max.col(y_test_pred)-1) == test$Y)

### Q4  Build 5 other classification models and compare with the 2 previous NN models.

In [50]:
# Train and test which was previously created was used



# Check the data
dim(train)
dim(test)

[1] 24000    25

[1] 6000   25

In [52]:
# Check the label proportion
round(table(train$Y) / nrow(train), 2)
round(table(test$Y) / nrow(test), 2)


   0    1 
0.78 0.22 


   0    1 
0.77 0.23 

In [54]:
# Define the ML classification task
train_task <- mlr::makeClassifTask(id ='dcc_train', data=train, target='Y')
test_task <- mlr::makeClassifTask(id='dcc_test', data=test, target='Y')

In [61]:
# Logistic Regression Lasso (l1)
set.seed(1)
learner <- mlr::makeLearner('classif.LiblineaRL1LogReg')  # Register a machine learning model
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.806

In [62]:
# Random Forest
set.seed(1)
learner <- makeLearner('classif.randomForest')
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.814

In [63]:
# k-Nearest Neighbor (k=50)
set.seed(1)
learner <- makeLearner('classif.knn', k=50)
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.7718333

In [64]:
# Adabag Boosting
set.seed(1)
learner <- makeLearner('classif.boosting')
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.8143333

In [65]:
# LDA (drop zero-variance features)
set.seed(1)
learner <- makeLearner('classif.lda')
model <- mlr::train(learner, filterFeatures(train_task, method='variance', threshold=0.1))
#model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.8076667

It seems the nueral networks overfit the data. According to the classification models Adabag Boosting and Random Foreset are performing well with almost the same accuracy of 0.814.
These two models are followed by LDA with 0.807 and Logistic Regression with accuracy of 0.806.
